# 1. 参展样机UVC灯管套件的设计

* 型号:GPH610T5L/HO
* 管直径:&oslash;15mm
* 安装长度:610mm
* 弧长:530mm
* 功率:55W
* UVC在254nm处强度:168uW/cm<sup>2</sup>
* UVC在254nm处输出功率:17.5W
* 寿命:16000hrs

In [2]:
import numpy as np

# 初始条件
lamp_power = 55  # 灯管功率，单位：瓦
efficiency = 0.3  # 灯管能量输出效率
UVC_output = lamp_power * efficiency  # 实际紫外线输出功率，单位：瓦
UVC_output_mW = UVC_output * 1000  # 转换为毫瓦

lamp_length = 53  # 灯管弧长，单位：cm

# 大肠杆菌99%消杀剂量，单位：mWs/cm²
required_dose = 6.6  # 消杀剂量

# 最小消杀时间，单位：秒
min_exposure_time = 2.5  # 秒

# 水流量，单位：立方米每小时
flow_rate_m3h = 10  # 立方米每小时
flow_rate_cm3s = flow_rate_m3h * 1e6 / 3600  # 转换为立方厘米每秒

# 石英套管和灯管寿命因素
quartz_transmittance = 0.9  # 石英套管透过率
lamp_lifetime_factor = 0.8  # 灯管寿命缩短后的效率
scaling_factor = 0.85  # 石英套管结垢后的透过率

# 综合效率计算
combined_efficiency = efficiency * quartz_transmittance * lamp_lifetime_factor * scaling_factor
UVC_output_combined_mW = lamp_power * combined_efficiency * 1000  # 综合效率下的紫外线输出功率，单位：mW

# 控制管道内径在100mm以内，并保证管道长度是灯管长度的整数倍
max_diameter = 8.2  # 管道最大内径，单位：cm

# 计算合理的管道直径和长度
def calculate_pipe_dimensions(flow_rate_cm3s, lamp_length, required_dose, UVC_output_combined_mW, min_exposure_time, max_diameter):
    for num_lamps in range(1, 11):
        total_length = lamp_length * num_lamps  # 总管道长度，单位：cm
        for diameter_cm in np.linspace(1, max_diameter, 1000):
            cross_section_area = np.pi * (diameter_cm / 2) ** 2  # 管道截面积，单位：cm²
            flow_speed = flow_rate_cm3s / cross_section_area  # 流速，单位：cm/s
            exposure_time = total_length / flow_speed  # 暴露时间，单位：秒

            if exposure_time >= min_exposure_time:
                intensity = UVC_output_combined_mW / (2 * np.pi * total_length * (diameter_cm / 2))  # 初始强度，单位：mW/cm²
                dose = intensity * exposure_time  # 剂量，单位：mWs/cm²
                if dose >= required_dose:
                    return diameter_cm, total_length, exposure_time, dose

    return None, None, None, None

# 计算合理的管道直径和长度
diameter, length, exposure_time, dose = calculate_pipe_dimensions(flow_rate_cm3s, lamp_length, required_dose, 
                                                                  UVC_output_combined_mW, min_exposure_time, max_diameter)

if diameter:
    print(f"合理的管道内径是: {diameter:.2f} cm")
    print(f"管道长度: {length:.2f} cm")
    print(f"暴露时间: {exposure_time:.2f} 秒")
    print(f"紫外线剂量: {dose:.2f} mWs/cm²")
else:
    print("找不到合理的管道内径和长度。")

合理的管道内径是: 7.46 cm
管道长度: 159.00 cm
暴露时间: 2.50 秒
紫外线剂量: 6.78 mWs/cm²
